In [1]:
import pandas as pd
pd.set_option('mode.chained_assignment', None)
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import os 
from rdkit.Chem import Lipinski
from rdkit.Chem import rdMolDescriptors
from rdkit.Chem import Descriptors, MolToSmiles, QED

import sys
sys.path.append("../sfi")  # Add the parent directory (project root) to the path
from logd_predictor_clean import LogDPredictor

In [2]:
# load the dataframes
rna_df_picked = pd.read_pickle('../data/diversity_picking/RNA/ECFP6_v2/rna_ECFP6_MaxMin_v3.pkl')
protein_df_picked = pd.read_pickle('../data/diversity_picking/ECFP6_v2/enamine_protein_ECFP6_MaxMin_v3.pkl').head(36937)


In [3]:
# using 'source' column separate rna_df_picked and protein_df_picked to datarames
enamine_protein = protein_df_picked[protein_df_picked['source'] == 'enamine_protein']
# change the 'smiles' column to 'SMILES' column
enamine_protein.rename(columns={'smiles':'SMILES'}, inplace=True)
chemdiv_rna = rna_df_picked[rna_df_picked['source'] == 'chemdiv']
enamine_rna = rna_df_picked[rna_df_picked['source'] == 'enamine']
life_chemicals_rna = rna_df_picked[rna_df_picked['source'] == 'life_chemicals']
robin_rna = rna_df_picked[rna_df_picked['source'] == 'robin']



# create folder for output propety_analysis
if not os.path.exists('../output/v2/property_analysis'):
   os.makedirs('../output/v2/property_analysis')



# create new dataframe with only the columns we need 'mol'
enamine_protein_mol = enamine_protein[['mol']]
chemdiv_rna_mol = chemdiv_rna[['mol']]
enamine_rna_mol = enamine_rna[['mol']]
life_chemicals_rna_mol = life_chemicals_rna[['mol']]
robin_rna_mol = robin_rna[['mol']]


In [4]:
# Combine datasets into one DataFrame
combined_df = pd.concat([enamine_protein, chemdiv_rna, enamine_rna, life_chemicals_rna, robin_rna])

# Sort datasets by size (largest to smallest)
datasets = [enamine_protein, chemdiv_rna, enamine_rna, life_chemicals_rna, robin_rna]
sorted_datasets = sorted(datasets, key=lambda x: x.shape[0], reverse=True)

# Finding all unique duplicates across datasets
all_duplicates = combined_df[combined_df.duplicated('SMILES', keep=False)]

# Remove duplicates in order from largest to smallest, except the smallest
for i in range(len(sorted_datasets) - 1):  # Skip the smallest dataset
    dataset = sorted_datasets[i]
    duplicates_to_remove = all_duplicates[~all_duplicates['source'].isin([dataset['source'].iloc[0]])]['SMILES']
    sorted_datasets[i] = dataset[~dataset['SMILES'].isin(duplicates_to_remove)]

# Extracting the updated datasets
enamine_protein, chemdiv_rna, enamine_rna, life_chemicals, robin_df = sorted_datasets

# Now the datasets are deduplicated in the desired order

In [5]:

# Assuming you have these dataframes already loaded: 
# enamine_protein, chemdiv_rna, enamine_rna, life_chemicals_rna, robin_rna

# Concatenate the dataframes into one
combined_df = pd.concat([
    enamine_protein, chemdiv_rna, enamine_rna, 
    life_chemicals_rna, robin_rna
])

# Finding duplicates in the 'SMILES' column
duplicates = combined_df[combined_df.duplicated('SMILES', keep=False)]

# Creating a summary of duplicates between each pair of datasets
summary = {}
for dataset1 in combined_df['source'].unique():
    for dataset2 in combined_df['source'].unique():
        if dataset1 != dataset2:
            pair = tuple(sorted([dataset1, dataset2]))
            if pair not in summary:
                duplicates_in_pair = duplicates[
                    (duplicates['source'] == dataset1) | 
                    (duplicates['source'] == dataset2)
                ]
                count = duplicates_in_pair['SMILES'].nunique()
                summary[pair] = count

# Displaying the summary
for pair, count in summary.items():
    print(f"Duplicates between {pair[0]} and {pair[1]}: {count}")


Duplicates between chemdiv and enamine_protein: 0
Duplicates between enamine and enamine_protein: 0
Duplicates between enamine_protein and life_chemicals: 0
Duplicates between enamine_protein and robin: 0
Duplicates between chemdiv and enamine: 0
Duplicates between chemdiv and life_chemicals: 0
Duplicates between chemdiv and robin: 0
Duplicates between enamine and life_chemicals: 0
Duplicates between enamine and robin: 0
Duplicates between life_chemicals and robin: 0


In [6]:
predict_logd = LogDPredictor(model_file_name='../sfi/model_plus.txt')


def calculate_properties(mol):
    """
    The `calculate_properties` function takes a molecule object as input and calculates various 
    molecular properties, such as molecular weight, heavy atom count, number of oxygen, nitrogen, 
    carbon, chlorine, fluorine, and sulfur atoms, hydrogen bond acceptors and donors, 
    number of rings, logarithm of the octanol-water partition coefficient (cLogP), 
    number of rotatable bonds, and topological polar surface area (TPSA). 
    The function returns a pandas Series containing the SMILES representation of 
    the molecule and the calculated properties.
    """
    smiles = MolToSmiles(mol, canonical=True)
    mw = Descriptors.MolWt(mol)
    heavy_atom_count = mol.GetNumHeavyAtoms()
    num_O_atoms = len([atom for atom in mol.GetAtoms() if atom.GetSymbol() == 'O'])
    num_N_atoms = len([atom for atom in mol.GetAtoms() if atom.GetSymbol() == 'N'])
    num_C_atoms = len([atom for atom in mol.GetAtoms() if atom.GetSymbol() == 'C'])
    num_Cl_atoms = len([atom for atom in mol.GetAtoms() if atom.GetSymbol() == 'Cl'])
    num_F_atoms = len([atom for atom in mol.GetAtoms() if atom.GetSymbol() == 'F'])
    num_S_atoms = len([atom for atom in mol.GetAtoms() if atom.GetSymbol() == 'S'])
    qed = QED.qed(mol)
    # use the predict_logd to calculate logd
    clogD = predict_logd.predict(mol) # model for this prediction can be found in /sfi/build_logd_model.ipynb

    hba = Lipinski.NumHAcceptors(mol)
    hbd = Lipinski.NumHDonors(mol)
    rings = rdMolDescriptors.CalcNumRings(mol)
    clogP = Descriptors.MolLogP(mol)
    n_rot = rdMolDescriptors.CalcNumRotatableBonds(mol)
    tpsa = rdMolDescriptors.CalcTPSA(mol)

    return pd.Series([smiles, mw, heavy_atom_count, num_O_atoms, num_N_atoms, num_C_atoms, 
                      num_Cl_atoms, num_F_atoms, num_S_atoms, hba, hbd, rings, clogP, n_rot, tpsa, qed, clogD],)


enamine_protein[['SMILES', 'MW', '#HeavyAtoms', 'NumO', 'NumN', 'NumC', 'NumCl', 'NumF', 'NumS', 'HBA', 'HBD', 'Rings', 'ClogP', '#RotBonds', 'TPSA', 'QED', 'ClogD']] = enamine_protein_mol.apply(lambda x: calculate_properties(x['mol']), axis=1)
chemdiv_rna[['SMILES', 'MW', '#HeavyAtoms', 'NumO', 'NumN', 'NumC', 'NumCl', 'NumF', 'NumS', 'HBA', 'HBD', 'Rings', 'ClogP', '#RotBonds', 'TPSA', 'QED', 'ClogD']] = chemdiv_rna_mol.apply(lambda x: calculate_properties(x['mol']), axis=1)
enamine_rna[['SMILES', 'MW', '#HeavyAtoms', 'NumO', 'NumN', 'NumC', 'NumCl', 'NumF', 'NumS', 'HBA', 'HBD', 'Rings', 'ClogP', '#RotBonds', 'TPSA', 'QED', 'ClogD']] = enamine_rna_mol.apply(lambda x: calculate_properties(x['mol']), axis=1)
life_chemicals_rna[['SMILES', 'MW', '#HeavyAtoms', 'NumO', 'NumN', 'NumC', 'NumCl', 'NumF', 'NumS', 'HBA', 'HBD', 'Rings', 'ClogP', '#RotBonds', 'TPSA', 'QED', 'ClogD']] = life_chemicals_rna_mol.apply(lambda x: calculate_properties(x['mol']), axis=1)
robin_rna[['SMILES', 'MW', '#HeavyAtoms', 'NumO', 'NumN', 'NumC', 'NumCl', 'NumF', 'NumS', 'HBA', 'HBD', 'Rings', 'ClogP', '#RotBonds', 'TPSA', 'QED', 'ClogD']] = robin_rna_mol.apply(lambda x: calculate_properties(x['mol']), axis=1)


In [7]:
# save to csv without index
enamine_protein.to_csv('../output/v2/property_analysis/enamine_protein.csv', index=False)
chemdiv_rna.to_csv('../output/v2/property_analysis/chemdiv_rna.csv', index=False)
enamine_rna.to_csv('../output/v2/property_analysis/enamine_rna.csv', index=False)
life_chemicals_rna.to_csv('../output/v2/property_analysis/life_chemicals.csv', index=False)
robin_rna.to_csv('../output/v2/property_analysis/robin_df.csv', index=False)

In [8]:
robin_rna.head(5)

,source,SMILES,mol,ECFP6,MW,#HeavyAtoms,NumO,NumN,NumC,NumCl,NumF,NumS,HBA,HBD,Rings,ClogP,#RotBonds,TPSA,QED,ClogD
32754,robin,C#CC1(N)CCCCC1,<rdkit.Chem.rdchem.Mol object at 0x7feae92dc450>,"[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",123.199,9,0,1,8,0,0,0,1,1,1,1.28120,0,26.02,0.482364,-0.415902
32043,robin,CSc1c(N(C)C)c(=O)c1=S,<rdkit.Chem.rdchem.Mol object at 0x7feae92dc4f0>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",187.289,11,1,1,7,0,0,2,4,0,1,1.43979,2,20.31,0.515274,1.928297
33318,robin,NCCCCNCCCN,<rdkit.Chem.rdchem.Mol object at 0x7feae92dc590>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",145.250,10,0,3,7,0,0,0,3,3,0,-0.33630,7,64.07,0.429544,-8.854660
33330,robin,S=P(N1CC1)(N1CC1)N1CC1,<rdkit.Chem.rdchem.Mol object at 0x7feae92dc6d0>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",189.224,11,0,3,6,0,0,1,1,0,3,0.15770,3,9.03,0.461380,-1.273761
32479,robin,Nc1ccc(-c2nc3nc(N)nc(N)c3nc2-c2ccc(N)cc2)cc1,<rdkit.Chem.rdchem.Mol object at 0x7feae92dc950>,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",344.382,26,0,8,18,0,0,0,8,4,4,2.08260,2,155.64,0.402617,1.444472


## Again with separated datasets not RNA and Protein picked but each is picked alone


In [9]:

enamine_protein = pd.read_pickle('../data/diversity_picking/ECFP6_v2/enamine_protein_ECFP6_MaxMin_v2.pkl')
chemdiv_rna = pd.read_pickle('../data/diversity_picking/ECFP6_v2/chemdiv_rna_ECFP6_MaxMin_v2.pkl')
enamine_rna = pd.read_pickle('../data/diversity_picking/ECFP6_v2/enamine_rna_ECFP6_MaxMin_v2.pkl')
life_chemicals = pd.read_pickle('../data/diversity_picking/ECFP6_v2/life_chemicals_rna_ECFP6_MaxMin_v2.pkl')
robin_df = pd.read_pickle('../data/diversity_picking/ECFP6_v2/robin_rna_ECFP6_MaxMin_v2.pkl')


# create new dataframe with only the columns we need 'mol' but just 2003 molecules from each dataset
enamine_protein_mol = enamine_protein[['mol']].head(2003)
chemdiv_rna_mol = chemdiv_rna[['mol']].head(2003)
enamine_rna_mol = enamine_rna[['mol']].head(2003)
life_chemicals_mol = life_chemicals[['mol']].head(2003)
robin_df_mol = robin_df[['mol']].head(2003)

In [10]:
# Load the datasets
enamine_protein = pd.read_pickle('../data/diversity_picking/ECFP6_v2/enamine_protein_ECFP6_MaxMin_v2.pkl')
chemdiv_rna = pd.read_pickle('../data/diversity_picking/ECFP6_v2/chemdiv_rna_ECFP6_MaxMin_v2.pkl')
enamine_rna = pd.read_pickle('../data/diversity_picking/ECFP6_v2/enamine_rna_ECFP6_MaxMin_v2.pkl')
life_chemicals = pd.read_pickle('../data/diversity_picking/ECFP6_v2/life_chemicals_rna_ECFP6_MaxMin_v2.pkl')
robin_df = pd.read_pickle('../data/diversity_picking/ECFP6_v2/robin_rna_ECFP6_MaxMin_v2.pkl')

# Combine datasets into one DataFrame
combined_df = pd.concat([enamine_protein, chemdiv_rna, enamine_rna, life_chemicals, robin_df])

# Sort datasets by size (largest to smallest)
datasets = [enamine_protein, chemdiv_rna, enamine_rna, life_chemicals, robin_df]
sorted_datasets = sorted(datasets, key=lambda x: x.shape[0], reverse=True)

# Finding all unique duplicates across datasets
all_duplicates = combined_df[combined_df.duplicated('smiles', keep=False)]

# Remove duplicates in order from largest to smallest, except the smallest
for i in range(len(sorted_datasets) - 1):  # Skip the smallest dataset
    dataset = sorted_datasets[i]
    duplicates_to_remove = all_duplicates[~all_duplicates['source'].isin([dataset['source'].iloc[0]])]['smiles']
    sorted_datasets[i] = dataset[~dataset['smiles'].isin(duplicates_to_remove)]

# Extracting the updated datasets
enamine_protein, chemdiv_rna, enamine_rna, life_chemicals, robin_df = sorted_datasets

# Now the datasets are deduplicated in the desired order


In [11]:
# Concatenate the dataframes into one
combined_df = pd.concat([
    enamine_protein, chemdiv_rna, enamine_rna, 
    life_chemicals, robin_df
])

# Finding duplicates in the 'SMILES' column
duplicates = combined_df[combined_df.duplicated('smiles', keep=False)]

# Creating a summary of duplicates between each pair of datasets
summary = {}
for dataset1 in combined_df['source'].unique():
    for dataset2 in combined_df['source'].unique():
        if dataset1 != dataset2:
            pair = tuple(sorted([dataset1, dataset2]))
            if pair not in summary:
                duplicates_in_pair = duplicates[
                    (duplicates['source'] == dataset1) | 
                    (duplicates['source'] == dataset2)
                ]
                count = duplicates_in_pair['smiles'].nunique()
                summary[pair] = count

# Displaying the summary
for pair, count in summary.items():
    print(f"Duplicates between {pair[0]} and {pair[1]}: {count}")

Duplicates between chemdiv and enamine_protein: 0
Duplicates between enamine and enamine_protein: 0
Duplicates between enamine_protein and life_chemicals: 0
Duplicates between enamine_protein and robin: 0
Duplicates between chemdiv and enamine: 0
Duplicates between chemdiv and life_chemicals: 0
Duplicates between chemdiv and robin: 0
Duplicates between enamine and life_chemicals: 0
Duplicates between enamine and robin: 0
Duplicates between life_chemicals and robin: 0


In [12]:
predict_logd = LogDPredictor(model_file_name='../sfi/model_plus.txt')


def calculate_properties(mol):
    """
    The `calculate_properties` function takes a molecule object as input and calculates various 
    molecular properties, such as molecular weight, heavy atom count, number of oxygen, nitrogen, 
    carbon, chlorine, fluorine, and sulfur atoms, hydrogen bond acceptors and donors, 
    number of rings, logarithm of the octanol-water partition coefficient (cLogP), 
    number of rotatable bonds, and topological polar surface area (TPSA). 
    The function returns a pandas Series containing the SMILES representation of 
    the molecule and the calculated properties.
    """
    smiles = MolToSmiles(mol, canonical=True)
    mw = Descriptors.MolWt(mol)
    heavy_atom_count = mol.GetNumHeavyAtoms()
    num_O_atoms = len([atom for atom in mol.GetAtoms() if atom.GetSymbol() == 'O'])
    num_N_atoms = len([atom for atom in mol.GetAtoms() if atom.GetSymbol() == 'N'])
    num_C_atoms = len([atom for atom in mol.GetAtoms() if atom.GetSymbol() == 'C'])
    num_Cl_atoms = len([atom for atom in mol.GetAtoms() if atom.GetSymbol() == 'Cl'])
    num_F_atoms = len([atom for atom in mol.GetAtoms() if atom.GetSymbol() == 'F'])
    num_S_atoms = len([atom for atom in mol.GetAtoms() if atom.GetSymbol() == 'S'])
    qed = QED.qed(mol)
    # use the predict_logd to calculate logd
    clogD = predict_logd.predict(mol)

    hba = Lipinski.NumHAcceptors(mol)
    hbd = Lipinski.NumHDonors(mol)
    rings = rdMolDescriptors.CalcNumRings(mol)
    clogP = Descriptors.MolLogP(mol)
    n_rot = rdMolDescriptors.CalcNumRotatableBonds(mol)
    tpsa = rdMolDescriptors.CalcTPSA(mol)

    return pd.Series([smiles, mw, heavy_atom_count, num_O_atoms, num_N_atoms, num_C_atoms, 
                      num_Cl_atoms, num_F_atoms, num_S_atoms, hba, hbd, rings, clogP, n_rot, tpsa, qed, clogD],)


enamine_protein_mol[['SMILES', 'MW', '#HeavyAtoms', 'NumO', 'NumN', 'NumC', 'NumCl', 'NumF', 'NumS', 'HBA', 'HBD', 'Rings', 'ClogP', '#RotBonds', 'TPSA', 'QED', 'ClogD']] = enamine_protein.apply(lambda x: calculate_properties(x['mol']), axis=1)
chemdiv_rna_mol[['SMILES', 'MW', '#HeavyAtoms', 'NumO', 'NumN', 'NumC', 'NumCl', 'NumF', 'NumS', 'HBA', 'HBD', 'Rings', 'ClogP', '#RotBonds', 'TPSA', 'QED', 'ClogD']] = chemdiv_rna.apply(lambda x: calculate_properties(x['mol']), axis=1)
enamine_rna_mol[['SMILES', 'MW', '#HeavyAtoms', 'NumO', 'NumN', 'NumC', 'NumCl', 'NumF', 'NumS', 'HBA', 'HBD', 'Rings', 'ClogP', '#RotBonds', 'TPSA', 'QED', 'ClogD']] = enamine_rna.apply(lambda x: calculate_properties(x['mol']), axis=1)
life_chemicals_mol[['SMILES', 'MW', '#HeavyAtoms', 'NumO', 'NumN', 'NumC', 'NumCl', 'NumF', 'NumS', 'HBA', 'HBD', 'Rings', 'ClogP', '#RotBonds', 'TPSA', 'QED', 'ClogD']] = life_chemicals.apply(lambda x: calculate_properties(x['mol']), axis=1)
robin_df_mol[['SMILES', 'MW', '#HeavyAtoms', 'NumO', 'NumN', 'NumC', 'NumCl', 'NumF', 'NumS', 'HBA', 'HBD', 'Rings', 'ClogP', '#RotBonds', 'TPSA', 'QED', 'ClogD']] = robin_df.apply(lambda x: calculate_properties(x['mol']), axis=1)


In [13]:
# save to csv without index
enamine_protein_mol.to_csv('../output/v2/property_analysis/enamine_protein_new.csv', index=False)
chemdiv_rna_mol.to_csv('../output/v2/property_analysis/chemdiv_rna_new.csv', index=False)
enamine_rna_mol.to_csv('../output/v2/property_analysis/enamine_rna_new.csv', index=False)
life_chemicals_mol.to_csv('../output/v2/property_analysis/life_chemicals_new.csv', index=False)
robin_df_mol.to_csv('../output/v2/property_analysis/robin_df_new.csv', index=False)
